In [1]:
import pandas as pd
import pybasilica.run as run

%load_ext autoreload
%autoreload 2

/Users/elenab/Library/r-miniconda-arm64/envs/basilica-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
m_path = "/Users/elenab/Library/CloudStorage/GoogleDrive-ele.buscaroli@gmail.com/My Drive/work/signatures/data_sigphylo.csv"
m = pd.read_csv(m_path)
ref_path = "/Users/elenab/GitHub/pybasilicah/pybasilica/data/cosmic/cosmic_catalogue.csv"
cosmic = pd.read_csv(ref_path, index_col=0)


In [ ]:
m 

In [ ]:
# import torch
# sig1 = torch.tensor(cosmic.values)[0]
# sig2 = torch.tensor(cosmic.values)[29]

# sig1[sig1==0] = 1e-7
# # print(torch.log(sig1))
# sig2[sig2==0] = 1e-7
# # print(torch.log(sig2))
# torch.nn.functional.kl_div(torch.log(sig1), torch.log(sig2), log_target=True, reduction="batchmean")


In [ ]:
m.shape 

In [66]:
one = run.fit(
    x=m, 
    k_list=[0,1,2], 
    lr=0.05, 
    n_steps=500, 
    enumer="parallel", 
    # cluster=[2], 
    # groups=[0,1,1,0,1], 
    beta_fixed=cosmic.iloc[:2,:], 
    hyperparameters={"alpha_var":10000},
    compile_model = False, 
    CUDA = False, 
    enforce_sparsity = False, 
    regularizer = "cosine", 
    reg_weight = 1., 
    reg_bic = True, 
    store_parameters=False, 
    verbose = False, 
    stage = "", 
    save_runs_seed=True,
    seed=[17],
    initializ_pars_fit = True,
    initializ_seed = False,
    new_hier=True
    )

# 5 = n_samples, 1,2 + 2 = n_sigs, 2 = n_groups

Running model with 0 signatures, None groups and 17 seed

line 882 0
line 900 10
Running model with 1 signatures, None groups and 17 seed

line 882 96
line 900 111
Running model with 2 signatures, None groups and 17 seed

line 882 192
line 900 212


In [67]:
obj = one[0]

In [68]:
obj.pi 

In [69]:
obj.groups 

In [70]:
obj.alpha

,SBS1,SBS2,D1,D2
0,0.264607,0.030306,0.682469,0.022618
1,0.256053,0.061681,0.589520,0.092747
2,0.023191,0.013614,0.065750,0.897444
3,0.018792,0.010357,0.037346,0.933505
4,0.149267,0.040111,0.670163,0.140459


In [24]:
obj.k_denovo

2

In [25]:
obj.k_fixed

2

In [26]:
obj.alpha_prior_unn

In [ ]:
obj.alpha_noise

In [ ]:
obj.alpha_unn + obj.alpha_noise 

In [ ]:
obj.alpha 

In [ ]:
from pyro import distributions as distr
import seaborn as sns

sns.displot(distr.HalfNormal(obj.hyperparameters["alpha_var"]).sample_n(1000).tolist())
sns.displot(distr.Exponential(obj.hyperparameters["exp_rate"]).sample_n(1000).tolist()) 
sns.displot(distr.Normal(1, 0.5).sample_n(1000).tolist())

In [ ]:
x2 = one.epsilon.astype(int)

In [ ]:
two = run.fit(
    x=x2, 
    k_list=[1,2,3,4,5], 
    lr=0.05, 
    n_steps=500, 
    enumer=False, 
    cluster=None, 
    groups=None, 
    beta_fixed=None, 
    compile_model = False, 
    CUDA = False, 
    enforce_sparsity = False, 
    regularizer = "cosine", 
    reg_weight = 1, 
    reg_bic = False, 
    store_parameters=False, 
    verbose=False, 
    stage = "two"
    )

In [ ]:
two.alpha

In [ ]:
import torch
import numpy as np

m1 = torch.tensor(m.values).float()
alpha1 = torch.tensor(one.alpha.values).float()
a1 = torch.matmul(torch.diag(torch.sum(m1, axis=1)), alpha1)

m2 = torch.tensor(x2.values).float()
alpha2 = torch.tensor(two.alpha.values).float()
a2 = torch.matmul(torch.diag(torch.sum(m2, axis=1)), alpha2)

b1 = torch.sum(a1, 1).unsqueeze(-1)
b2 = torch.sum(a2, 1).unsqueeze(-1)
b3 = b1 + b2

alpha1 = a1 / b3
alpha2 = a2 / b3

In [ ]:
sample_names = list(one.alpha.index)
fixed_names = list(one.alpha.columns)
denovo_names = list(two.alpha.columns)
alpha1 = pd.DataFrame(np.array(alpha1), index=sample_names, columns=fixed_names)
alpha2 = pd.DataFrame(np.array(alpha2), index=sample_names, columns=denovo_names)
alpha = pd.concat([alpha1, alpha2], axis=1)

In [ ]:
alpha

In [ ]:
torch.sum(torch.tensor(alpha.values).float(), 1).unsqueeze(-1)